In [1]:
import os

import torch
from cgan import Generator, Discriminator
from utils import create_dataloader

/global/home/users/xju/miniconda3/envs/pyG-1.12.1/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

data_path = "/global/home/users/xju/projects/herwig/data"
filename = os.path.join(data_path, "allHadrons_mode4_with_quark_with_pert.npz")

In [4]:
batch_size = 10
num_workers = 2

train_loader, test_loader, xlabels = create_dataloader(filename, batch_size, num_workers)

(7809, 2) (7809, 8)


In [7]:
batch = next(iter(train_loader))

In [14]:
num_layers = 2
hidden_dim = 64
noise_dim = 16

batch = next(iter(train_loader))

cond_dim = batch[0].shape[1]
output_dim = batch[1].shape[1]

In [15]:
gene = Generator(noise_dim, cond_dim, output_dim, num_layers, hidden_dim)
disc = Discriminator(output_dim, cond_dim, num_layers, hidden_dim)

TypeError: __init__() got an unexpected keyword argument 'activation'